# Explaining heart disease


In [ ]:
from explainer.explainers import feature_attributions_explainer

import warnings
warnings.filterwarnings('ignore')

In [ ]:
feature_attributions_explainer.entry_points

In [ ]:
from explainer.explainers import metrics_explainer
metrics_explainer.entry_points

In [ ]:
import tensorflow as tf
import pandas as pd

tf.__version__

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
df = pd.read_csv(file_url)

In [ ]:
# make target variable
y = df.pop('target')

In [ ]:
# prepare features
list_numerical = ['age', 'thalach', 'trestbps',  'chol', 'oldpeak']

X = df[list_numerical]

## Data Splitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Feature Preprocessing

In [ ]:
scaler = StandardScaler().fit(X_train[list_numerical]) 

X_train[list_numerical] = scaler.transform(X_train[list_numerical])
X_test[list_numerical] = scaler.transform(X_test[list_numerical])

## Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

In [ ]:
model.compile(optimizer="adam", 
              loss ="binary_crossentropy", 
              metrics=["accuracy"])

model.fit(X_train, y_train, 
         epochs=15, 
         batch_size=13,
         validation_data=(X_test, y_test)
         )

## Visualize the connectivity graph:

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")


## Accuracy

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

print("Accuracy", accuracy)

In [ ]:
predictions = model.predict(X_train)

In [ ]:
print(
    "This particular patient had a %.1f percent probability "
    "of having a heart disease, as evaluated by our model." % (100 * predictions[0][0],)
)

In [ ]:
import shap
shap.initjs()
explainer = shap.KernelExplainer(model, X_train.iloc[:50,:])
shap_values = explainer.shap_values(X_train.iloc[20,:], nsamples=500)
shap.force_plot(explainer.expected_value, shap_values[0], X_train.iloc[20,:])

In [ ]:
kernelexplainer = feature_attributions_explainer['kernelexplainer'](model, X_train)

In [ ]:
kernelexplainer.visualize(X_train.iloc[20,:])